### Implementation of the paper "Bias Estimation and Gravity Compensation For Force-Torque Sensors"

$\mathbf{F}_s = \mathbf{F}_b + \mathbf{F}_g + \mathbf{F}_n$

### Imports and helper functions

In [8]:
import numpy as np

def rotation_from_euler(a, b, g):
    Rx=np.array([[1, 0, 0], [0, np.cos(a), -np.sin(a)], [0, np.sin(a), np.cos(a)]], dtype=float)
    Ry=np.array([[np.cos(b), 0, np.sin(b)], [0, 1, 0], [-np.sin(b), 0, np.cos(b)]], dtype=float)
    Rz=np.array([[np.cos(g), -np.sin(g), 0], [np.sin(g), np.cos(g), 0], [0, 0, 1]], dtype=float)
    return np.dot(Rz, np.dot(Ry, Rx))

### Unknown parameters to estimate

In [16]:
m=0.25
r=np.array([-0.9, -0.1, 7.1])/1000.0 # centre of mass
Fb = np.array([0.3128, -1.0857, 4.8458, 0.0006, 0.1732, -0.0431]) # FT bias

### Gravity compensation $\mathbf{F}_g$

In [19]:
def ft_gravity_compensation(mass, position, orientation, gravity=np.array([0., 0., -9.81])):
    """ F_g
        mass: (1 x 1) mass of the ft sensor and any additional attached element.
        position: (3 x 1) center of mass 
        orientation: (3 x 3) orientation of the tool and ft sensor with respect to the inertial frame
        gravity: (3 x 1) 
    """
    force=m*np.dot(np.transpose(orientation), gravity)
    moment=np.cross(m*position, np.dot(np.transpose(orientation), gravity))
    return np.array([force[0], force[1], force[2], moment[0], moment[1], moment[2]])

### Virtual FT sensor

In [20]:
FTS = lambda orientation : ft_gravity_compensation(m, r, orientation) + Fb

### Estimate bias $\mathbf{F}_b$

In [75]:
g  = np.array([0, 0, -9.81])
R1 = rotation_from_euler(0, 0, 0)
R2 = rotation_from_euler(np.pi, 0, 0)
R3 = rotation_from_euler(0, np.pi/2.0, 0)
R4 = rotation_from_euler(0, -np.pi/2.0, 0)
R5 = rotation_from_euler(np.pi/2.0, 0, 0)
R6 = rotation_from_euler(-np.pi/2.0, 0, 0)

# gravity force down z axis
z1 = FTS(R1)
z2 = FTS(R2)
g1 = np.dot(np.transpose(R1), g)
g2 = np.dot(np.transpose(R2), g)

# gravity force down x axis
z3 = FTS(R3)
z4 = FTS(R4)
g3 = np.dot(np.transpose(R3), g)
g4 = np.dot(np.transpose(R4), g)

# gravity force down y axis
z5 = FTS(R5)
z6 = FTS(R6)
g5 = np.dot(np.transpose(R5), g)
g6 = np.dot(np.transpose(R6), g)

hFb = (z1+z2+z3+z4+z5+z6)/6.0

print('Estimated Fb {0}'.format(np.round(hFb, 2)))

Estimated Fb [ 0.31 -1.09  4.85  0.    0.17 -0.04]


### Estimate mass

In [74]:
FT = np.vstack((z1, z2, z3, z4, z5, z6)) - hFb
F = FT[:, :3]
G = np.vstack((g1, g2, g3, g4, g5, g6))

invGG = np.linalg.pinv(np.dot(np.transpose(G), G)) 
hM = np.dot(invGG, np.dot(np.transpose(G), F))
hm = hM[0, 0]
np.round(hm, 4)
print('Estimated mass: {}'.format(hm))

Estimated mass: 0.25


### Estimate center of mass